In [1]:
!pip install --quiet boto3
!pip install --quiet sagemaker
!pip install --quiet imbalanced-learn
!pip install --quiet xgboost


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import json

In [3]:
train = pd.read_csv('test.csv', index_col=0)
test  = pd.read_csv('test.csv', index_col=0)

In [4]:
train.head(2)

,person,time,amount,gender,age,income,reward,difficulty,duration,offer_type,email,social,mobile,web,days_since_membership,offered_channels_count,event
offer id,,,,,,,,,,,,,,,,,
9b98b8c7a33c4b65b9aebfe6a799e6d9,6569,276,15.77,M,45,94000.0,5.0,5.0,7.0,bogo,1,0,1,1,524,3,transaction
fafdcd668e3743c1bb461111dcafc2a4,9759,60,16.03,M,60,73000.0,2.0,10.0,10.0,discount,1,1,1,1,1092,4,transaction


In [5]:
with open('event_to_labels.json', 'r') as p:
    event_to_labels = json.load(p)

In [6]:
event_to_labels

{'offer completed': 0,
 'offer received': 1,
 'offer viewed': 2,
 'transaction': 3}

In [7]:
target = 'event'

X_train = train.drop(columns=target)
y_train = train.get(target).map(event_to_labels)

X_test = train.drop(columns=target)
y_test = train.get(target).map(event_to_labels)

In [8]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, precision_score, classification_report

np.random.seed(42)

In [9]:
numeric_features = make_column_selector(dtype_include=np.number)
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)), 
        ("scaler", StandardScaler())
    ]
)

categorical_features = make_column_selector(dtype_exclude=np.number)
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)



ros = RandomOverSampler(random_state=42, sampling_strategy='auto')
rus = RandomUnderSampler(random_state=42, sampling_strategy='auto')
bbc = XGBClassifier(random_state=42)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("oversample", ros),
        ("undersample", rus),        
        ("model", bbc),
    ]
)

In [10]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f674ea43a60>),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=F...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [11]:
y_pred = np.argmax(model.predict_proba(X_train) > 0.47, axis=1)

In [12]:
pd.DataFrame(classification_report(y_train, y_pred, labels=list(event_to_labels.values()), target_names=list(event_to_labels.keys()), output_dict=True)).T

,precision,recall,f1-score,support
offer completed,0.270332,0.890102,0.414712,5314.000000
offer received,0.677598,0.596523,0.634481,9837.000000
offer viewed,0.625260,0.828471,0.712662,9083.000000
transaction,0.904364,0.348010,0.502610,22689.000000
accuracy,0.554504,0.554504,0.554504,0.554504
macro avg,0.619388,0.665776,0.566116,46923.000000
weighted avg,0.730994,0.554504,0.560962,46923.000000


In [13]:
y_pred = np.argmax(model.predict_proba(X_test) > 0.47, axis=1)

In [14]:
pd.DataFrame(classification_report(y_test, y_pred, labels=list(event_to_labels.values()), target_names=list(event_to_labels.keys()), output_dict=True)).T

,precision,recall,f1-score,support
offer completed,0.270332,0.890102,0.414712,5314.000000
offer received,0.677598,0.596523,0.634481,9837.000000
offer viewed,0.625260,0.828471,0.712662,9083.000000
transaction,0.904364,0.348010,0.502610,22689.000000
accuracy,0.554504,0.554504,0.554504,0.554504
macro avg,0.619388,0.665776,0.566116,46923.000000
weighted avg,0.730994,0.554504,0.560962,46923.000000


#### Emulation of the predictor

In [15]:
def model_fn(model):
    # fake model_fn
    return model

def input_fn(request_body, request_content_type):
    print('Start input function')
    if request_content_type=='text/csv':
        data = pd.read_csv(io.StringIO(request_body))
        return data
    if request_content_type=='application/json':
        data = pd.read_json(json.loads(request_body), orient='records')
        return data
    if request_content_type=='application/x-parquet':
        data = pd.read_parquet(io.BytesIO(request_body))
        return data
    raise ValueError('Accept header must be application/json or text/csv or application/x-parquet')

        
def predict_fn(input_object, model):
    print('Start predict function')
    if len(input_object.shape) <= 1:
        input_object = input_object.to_frame().T
    columns = ['offer completed', 'offer received', 'offer viewed', 'transaction']
    predictions = pd.DataFrame(model.predict_proba(input_object), index=input_object.index, columns=columns)
    print('End predict function')
    return predictions

def output_fn(predictions, content_type):
    print('Start output function')
    if content_type=='application/json' or content_type=='application/x-parquet' or content_type=='text/csv':
        result = json.dumps(predictions.to_dict())
        return result
    print('End Output function')

In [16]:
import joblib
import json

model = model_fn(model)

In [17]:
samples = X_train.sample(1).to_json(orient='records')

#samples = X_train.sample(1).to_json(orient='records', lines=True)
data = input_fn(request_body=json.dumps(samples), request_content_type='application/json')
payload = json.dumps(samples)

Start input function


/tmp/ipykernel_4289/2259011953.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(json.loads(request_body), orient='records')


In [18]:
predictions = predict_fn(data, model)

Start predict function
End predict function


In [19]:
pd.read_json(output_fn(predictions, content_type='application/json'))

Start output function


/tmp/ipykernel_4289/3124464824.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_json(output_fn(predictions, content_type='application/json'))


,offer completed,offer received,offer viewed,transaction
0,0.00387,0.242544,0.61806,0.135526
